In [1]:
import numpy as np
import pandas as pd
import os
import re
import math
import jenspy as jk

In [95]:
#熵值法小程序
def deal_data(df_select):
    for i in range(len(df_select.columns)):
        #print(i)
        k=pd.to_numeric(df_select.iloc[:,i])
        #k=np.log(k)
        if min(k)<=0:
            df_select.iloc[:,i]=pd.to_numeric(df_select.iloc[:,i])-min(k)+1
        else:
            df_select.iloc[:,i]=k
    return df_select
def proportionMax(x):
    P = x/np.max(x)
    p = P/sum(P)
    return p
def proportionMin(x):
    P = np.min(x)/x 
    p = P/sum(P)
    return p
def EntropyMax(x):
    p=proportionMax(x)    
    k = 1/(math.log(len(x)))  #单值
    data_log = np.log(p)
    data_log=list(data_log)
    p=np.matrix(p)
    data_log=np.matrix(data_log)
    entropy = p*data_log.T
    entropy=entropy.tolist()
    entropy=entropy[0][0]
    e = -k*entropy
    return e
def EntropyMin(x):
    p=proportionMin(x) 
    k = 1/(math.log(len(x)))  #单值
    data_log = np.log(p)
    data_log=list(data_log)
    p=np.matrix(p)
    data_log=np.matrix(data_log)
    entropy = p*data_log.T
    entropy=entropy.tolist()
    entropy=entropy[0][0]
    e = -k*entropy
    return e
def change(x):
    inter=(x-min(x))/(max(x)-min(x))
    p=inter*100
    return p
#直接对columns使用（apply到Dataframe上）
#这里对求出来的entropy使用
def valuate(proportion,Entropy,deal='normal'):
    g = 1-Entropy
    a = g/sum(g)
    if deal=='log':
        v = [np.log(sum(np.array(proportion.loc[i])*a)) for i in range(len(proportion))]
    elif deal=='normal':
        v = [sum(np.array(proportion.loc[i])*a) for i in range(len(proportion))]
    return v

def entropy(df_select,num=2,Min=[],minmax='max',deal='normal',number=False):
#   df_select=df_select.fillna(int(0))
    df_select=deal_data(df_select)
    df_select.index=range(len(df_select))
    v_all = []
    ee_all = []
    pp_all = []
    k = []
    for i in range(0,len(df_select.columns),num):
        df=pd.DataFrame(df_select.ix[:,i:i+num])
        if minmax=='max':
            ee=df.apply(EntropyMax)
            pp=df.apply(proportionMax)
##            print(ee)
        elif minmax=='min':
            ee=df.apply(EntropyMin)
            pp=df.apply(proportionMin)
##            print(ee)
        else:
            small=[]
            for a in df.columns:
                for kk in Min:
                    if a.find(kk)!=-1:
                        small.append(a)
                        break
            ee=df.apply(EntropyMax)
            pp=df.apply(proportionMax)
            for name in small:
                ee[name]=EntropyMin(df[name])
                pp[name]=proportionMin(df[name])
        v = valuate(pp,ee,deal=deal)      
        ee_all.append(ee)
        pp_all.append(pp)
        v_all.append((np.array(v)).tolist())
        k.append(df_select.columns[i])
    KK=pd.DataFrame(v_all).T
    if number:
        KK=KK.apply(change)
        KK=pd.DataFrame(KK)
    KK.columns=[str(i)+'得分' for i in k]
    return KK,ee_all,pp_all

In [3]:
firm=[]
os.chdir("./数据")
path=os.getcwd()
for i in os.listdir(path):
    if '.' not in i:
        firm.append(i)

In [106]:
import shutil
name=firm[0]
os.chdir(os.path.join(path,name))
df1=pd.read_excel('财务数据比较.xlsx',index_col='代码')
df2=pd.read_excel('财务比率比较.xlsx',index_col='代码')

In [107]:
df=df1.join(df2)
df=df.dropna()
columns=df.columns
col=[[],[],[],[]]
for i in columns:
    if i=='证券简称':
        continue
    elif int(i[:4])==2016:
        col[0].append(i)
    elif int(i[:4])==2017:
        col[1].append(i)
    elif int(i[:4])==2018:
        col[2].append(i)
    elif int(i[:4])==2019:
        col[3].append(i)
df=df.replace('--',0)

In [108]:
import jenkspy as jk
result=[]
jenk=[]
for i in range(len(col)):
    test,ee,pp=entropy(df[col[i]],num=len(col[i]),Min=[],minmax='max',deal='log',number=True)
    test.columns=['%s年报得分'%str(2016+i)]
    score=test['%s年报得分'%str(2016+i)].tolist()
    jenks=jk.jenks_breaks(score,5)
    jenk.append(jenks)
    cate=[]
    for i in score:
        for j in range(1,len(jenks)):
            if i <=jenks[j]:
                cate.append(j)
                break
    test[test.columns[0]+'类别']=cate
    test.index=df.index
    df=df.join(test)

C:\Users\yaoyi\Miniconda3\lib\site-packages\ipykernel_launcher.py:68: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [109]:
df.to_excel('财务数据得分.xlsx')

In [110]:
test.describe()

,2019年报得分,2019年报得分类别
count,305.000000,305.000000
mean,40.189266,2.606557
std,16.547846,1.179299
min,0.000000,1.000000
25%,28.038427,2.000000
50%,38.133534,3.000000
75%,50.115658,3.000000
max,100.000000,5.000000


In [99]:
jenk

[[0.0, 17.293178456973056, 33.153702899520816, 53.86767968362185, 100.0],
 [0.0, 18.096166647905594, 34.54934771425393, 55.6897334279564, 100.0],
 [0.0, 33.21927426762112, 46.18459600349529, 62.85874206682664, 100.0],
 [0.0, 30.83271631333443, 47.31769640272898, 65.75714331950002, 100.0]]